# Patient Reviewer

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from JupyterReviewer.Reviewers.PatientReviewer import PatientReviewer
from JupyterReviewer.Reviewers.PatientReviewer import collect_data
from JupyterReviewer.Data import DataAnnotation
from JupyterReviewer.ReviewDataApp import AppComponent

import pandas as pd
from dash import html

## Edit the code below according to the commented directions

### Generate a participant and a sample file

In [176]:
samples_df, participants_df = collect_data(
    '../../example_notebooks/example_data/example_patient_reviewer_paired_clean_input.yaml'
)

In [224]:
samples_df

,sample_id,cnv_seg_fn,participant_id,collection_date_dfd,preservation_method,wxs_ploidy,wxs_purity
0,06213_101_BB_090712,gs://fc-7e497e6c-57f5-4577-a386-e913b4ccbdaa/9...,ONC110293,14,NA,1.83,0.18
1,06213_110_BB_071213,gs://fc-7e497e6c-57f5-4577-a386-e913b4ccbdaa/9...,ONC123758,270,NA,2.64,0.044
2,06213_112_BB_110513,gs://fc-7e497e6c-57f5-4577-a386-e913b4ccbdaa/9...,ONC130272,34,NA,4.05,0.197
3,06213_15_BB_061908,gs://fc-7e497e6c-57f5-4577-a386-e913b4ccbdaa/9...,ONC7299,0,NA,1.78,0.21
5,06213_15_BB_072208,gs://fc-7e497e6c-57f5-4577-a386-e913b4ccbdaa/9...,ONC7299,33,NA,1.77,0.23
...,...,...,...,...,...,...,...
113,BRCA-63-Tumor-SM-4JPBE,gs://fc-a984e71a-8456-4f8d-a8e4-3185f88359cd/f...,ONC69869,402,NaN,3.86,0.41
115,BRCA-89-Tumor-SM-4JPTW,gs://fc-a984e71a-8456-4f8d-a8e4-3185f88359cd/f...,ONC1580,239,NaN,3.284,0.279
117,BRCA-BOT2067-Tumor-SM-35BD2,gs://fc-a984e71a-8456-4f8d-a8e4-3185f88359cd/f...,ONC68035,17,NaN,1.948,0.579
119,BRCA-BOT2110-Tumor-SM-35BCZ,gs://fc-a984e71a-8456-4f8d-a8e4-3185f88359cd/f...,ONC97091,31,NaN,2.437,0.124


### Create patient reviewer object

In [194]:
na_values = ['#N/A', '#N/A N/A', '#NA', '-1.#IND', '-1.#QNAN', '-NaN', '-nan', '1.#QNAN', '<NA>', 'N/A', 'NULL', 'NaN', 'n/a', 'nan', 'null', '', 'unknown', 'not reported']
reviewer = PatientReviewer()
reviewer.set_review_data(
    # Enter the file path where you would like your pkl file of your review data to be stored
    data_pkl_fn='../../Reviewer.pkl',
    # Breifly describe the source of your data and its purpose
    description='first review',
    # Enter the file path to your data to be reviewed
    participant_df=pd.read_csv('../../example_notebooks/example_data/paired_analysis_clean/paired_analysis_clean_participants.txt', sep='\t', na_values=na_values, keep_default_na=False).set_index('participant_id'),
    sample_df=pd.read_csv('../../example_notebooks/example_data/paired_analysis_clean/paired_analysis_clean_samples.txt', sep='\t', na_values=na_values, keep_default_na=False).set_index('sample_id'),
    preprocess_data_dir = '../../example_notebooks/example_data/preprocess_data'
)

reviewer.set_review_app(preprocess_data_dir = '../../example_notebooks/example_data/preprocess_data', drivers_fn='drivers.csv')
reviewer.set_default_review_data_annotations_configuration()
reviewer.set_default_autofill()


cnv figs directory already exists: ../../example_notebooks/example_data/preprocess_data/cnv_figs


/Users/svanseters/Broad/JupyterReviewer/JupyterReviewer/ReviewData.py:38: UserWarning:

Loading existing data pkl file



### Pass custom column color assignments into the review app with the following syntax:

In [16]:
# reviewer.set_review_app(custom_colors=[
#     [column_id_1, filter_query_1, text_color_1, background_color_1], 
#     [<same for all additional columns/queries>]
# ])

#Example (comment out reviewer.set_review_app() above and uncomment below to try):

# reviewer.set_review_app(custom_colors=[
#     ['Variant_Classification', 'Silent', 'Black', 'LightBlue'], 
#     ['Variant_Classification', 'Missense_Mutation', 'White', 'Indigo']
# ])

### Add your own components below 

In [17]:
# reviewer.app.add_component(AppComponent('component name', layout)))

### Add your own annotations below

In [8]:
# reviewer.review_data.add_annotation('annot name', ReviewDataAnnotation())

## Run

In [195]:
reviewer.run(mode='external', port=8089)

Dash app running on http://0.0.0.0:8089/


/Users/svanseters/Broad/JupyterReviewer/reviewEnv/lib/python3.9/site-packages/google/auth/_default.py:79: UserWarning:

Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/

/Users/svanseters/Broad/JupyterReviewer/JupyterReviewer/AppComponents/CNVPlotComponent.py:269: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/Users/svanseters/Broad/JupyterReviewer/JupyterReviewer/AppComponents/CNVPlotComp